In [ ]:
pip install python-terrier


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 2.8 MB/s 
     |████████████████████████████████| 1.1 MB 21.7 MB/s 
     |████████████████████████████████| 69 kB 6.3 MB/s 
     |████████████████████████████████| 303 kB 36.9 MB/s 
     |████████████████████████████████| 46 kB 3.4 MB/s 
     |████████████████████████████████| 45 kB 2.5 MB/s 
     |████████████████████████████████| 294 kB 35.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.2 MB 45.4 MB/s 
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 126 kB 53.3 MB/s 
     |████████████████████████████████| 285 kB 40.2 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.8.1-py3-none-any.whl size=104091 sha256=0101ddfc7ebae63b68241f5b0d185788777d8b64bfd2b

In [ ]:
import pyterrier as pt
pt.init()


terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.8.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading the indexes for Podcast Small Test

In [ ]:
# Raw Document Index 
sample_set_we = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/sample_without_test/data.properties")
raw= pt.IndexFactory.of(sample_set_we)

In [ ]:
#Wikipedia Index 
wiki_instance = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raws_test/data.properties")
wiki_no_difference= pt.IndexFactory.of(wiki_instance)


In [ ]:
#Wikipedia Index freq 2
wiki_instance_2 = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raw_freq_2_test/data.properties")
wiki_no_difference_2= pt.IndexFactory.of(wiki_instance_2)

In [ ]:
#Wikipedia Index freq 5
wiki_instance_5 = pt.IndexRef.of("/content/drive/MyDrive/IRDM/xml_dir/wikified_index_raw_freq_5_test/data.properties")
wiki_no_difference_5= pt.IndexFactory.of(wiki_instance_5)

In [ ]:
import pandas as pd
id_template = 'spotify:episode:{}_{}'

def get_ep(a,b):
  return id_template.format(a,b)

DCU Approach (Raw Index + Entities + BM25 + DPH)




In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(raw, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(raw) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()

combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.38016443048083853,
 'ndcg': 0.5939376735955048,
 'ndcg_cut_30': 0.41829745888382824,
 'P_10': 0.49791666666666684}

Raw Index + Original Topics  + BM25 + DPH




In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(raw, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(raw) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()

combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.3632304809446112,
 'ndcg': 0.5735183947465338,
 'ndcg_cut_30': 0.39374726956189665,
 'P_10': 0.47083333333333344}

Raw Index + Wikfied Queries + BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_tests.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(raw, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(raw) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()

combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.2361849458461869,
 'ndcg': 0.486370806747003,
 'ndcg_cut_30': 0.24698141820518424,
 'P_10': 0.27291666666666664}

Raw Index + Entities + QE Approach

In [ ]:

qe = pt.BatchRetrieve(raw, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(raw, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(raw, controls={'wmodel': "PL2","c":20})


topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")


import pandas as pd
id_template = 'spotify:episode:{}_{}'
res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

13:09:20.538 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.3971810164272578, 'ndcg': 0.5993524839324625, 'ndcg_cut_30': 0.4275762502665781, 'P_10': 0.51875}


Wikfied Index + Wikified Queries + BM25 +DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_tests.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()


import pandas as pd
id_template = 'spotify:episode:{}_{}'
combined= bm25 + dph
res=combined.transform(topics)

def get_ep(a,b):
  return id_template.format(a,b)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("DCU Linear Combination Minus Entities ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

DCU Linear Combination Minus Entities 
{'map': 0.31938469425764465, 'ndcg': 0.5601426167843774, 'ndcg_cut_30': 0.37249486437010676, 'P_10': 0.42916666666666664}


Raw Index + Wikified Queries + QE approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_tests.xml",format="trecxml")


qe = pt.BatchRetrieve(raw, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(raw, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(raw, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

13:11:02.093 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.2559808251173357, 'ndcg': 0.4870956748156814, 'ndcg_cut_30': 0.25452680738721034, 'P_10': 0.29375}


Raw Index + Original Topics + QE approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")


qe = pt.BatchRetrieve(raw, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(raw, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(raw, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

13:12:21.726 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.37359149332068187, 'ndcg': 0.5755390063437716, 'ndcg_cut_30': 0.3979215958035478, 'P_10': 0.47708333333333347}


Raw Index + Nouns and Entities + QE approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth_test.xml",format="trecxml")


qe = pt.BatchRetrieve(raw, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(raw, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(raw, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

22:49:08.426 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.3671503383056933, 'ndcg': 0.5844762075103843, 'ndcg_cut_30': 0.40052623235298485, 'P_10': 0.4541666666666668}


Raw Index + Nouns + QE approach

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns_test_2.xml",format="trecxml")


qe = pt.BatchRetrieve(raw, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(raw, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(raw, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

22:49:58.204 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.3651404632198753, 'ndcg': 0.5818792167629907, 'ndcg_cut_30': 0.39562134226467105, 'P_10': 0.44583333333333336}


Raw Index + Entities + PL2 

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

pl2= pt.BatchRetrieve(raw, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])



res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])

df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]


rsl=df[["qid","docno","rank","score"]]



qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)

print("PL 2 ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))


PL 2 
{'map': 0.38092245405721964, 'ndcg': 0.5928752340729185, 'ndcg_cut_30': 0.4166458470954472, 'P_10': 0.5000000000000002}


Wikified Index + Original Topics + BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()



combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.351904977747342,
 'ndcg': 0.5726544417005525,
 'ndcg_cut_30': 0.38470239306357534,
 'P_10': 0.4854166666666668}

Wikified Index + Wikfied Query + Entities + BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_test_ent.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()



combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.32322613508062265,
 'ndcg': 0.5645330118335431,
 'ndcg_cut_30': 0.37791726271009957,
 'P_10': 0.4395833333333334}

Wikified Index + Nouns + Entities + BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth_test.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()



combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.3024268947257042,
 'ndcg': 0.5545293155275062,
 'ndcg_cut_30': 0.3526854971772618,
 'P_10': 0.4041666666666666}

Wikified Index + Entities + BM25 + DPH

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()



combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.36875666113489886,
 'ndcg': 0.5938723079062809,
 'ndcg_cut_30': 0.4096767528975363,
 'P_10': 0.5000000000000002}

Wikified Index Freq 2 + Entities + BM25 + DPH

In [ ]:

topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference_2, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference_2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()



combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.34167646100573595,
 'ndcg': 0.574869446908742,
 'ndcg_cut_30': 0.3912329682498769,
 'P_10': 0.46875000000000017}

Wikified Index + Entities + PL2

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

pl2= pt.BatchRetrieve(wiki_no_difference, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.37236685267689823,
 'ndcg': 0.5976718834341415,
 'ndcg_cut_30': 0.4249786155986821,
 'P_10': 0.5000000000000001}

Wikified Index + Wikfied Query + Entities + PL2

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/wikified_query_test_ent.xml",format="trecxml")

pl2= pt.BatchRetrieve(wiki_no_difference, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.24207505807010596,
 'ndcg': 0.48642307732611073,
 'ndcg_cut_30': 0.24770307784711884,
 'P_10': 0.23124999999999998}

Wikfied Index + Original Topics + PL2 

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")

pl2= pt.BatchRetrieve(wiki_no_difference, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])

res=pl2.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.35324086720528775,
 'ndcg': 0.5757862883297452,
 'ndcg_cut_30': 0.40162470018075624,
 'P_10': 0.4833333333333334}

Wikfied Index + Entities + QE Approach 

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")


qe = pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(wiki_no_difference, fb_terms=10, fb_docs=6) >> \
            pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

13:15:21.035 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.36304626792569433, 'ndcg': 0.5864968563572276, 'ndcg_cut_30': 0.4126850232069843, 'P_10': 0.5291666666666666}


Wikfied Index + Entities + QE Approach 

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")


qe = pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(wiki_no_difference, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

13:16:40.885 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.37037308424229415, 'ndcg': 0.5937695302259492, 'ndcg_cut_30': 0.41938373169627, 'P_10': 0.5208333333333334}


Wikfied Index + Original Topics + QE Approach 

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/podcasts_2020_topics_test.xml",format="trecxml")


qe = pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(wiki_no_difference, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

13:17:56.958 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.3594896727893227, 'ndcg': 0.5818676337919272, 'ndcg_cut_30': 0.40024616726520473, 'P_10': 0.4729166666666667}


Wikfied Index + Nouns + QE Approach 

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns_test_2.xml",format="trecxml")


qe = pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(wiki_no_difference, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

13:19:35.735 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.3241351734246641, 'ndcg': 0.5533233582039843, 'ndcg_cut_30': 0.3552999777173704, 'P_10': 0.41875}


Wikfied Index + Nouns + Entities + QE Approach 

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/noun_ent_bth_test.xml",format="trecxml")


qe = pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(wiki_no_difference, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "PL2","c":20})

res=qe.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])


df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

rsl=df[["qid","docno","rank","score"]]


qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
print("QE ")
print(pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"]))

13:21:18.014 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1
QE 
{'map': 0.32015917427957274, 'ndcg': 0.5508145410640551, 'ndcg_cut_30': 0.34520152183061475, 'P_10': 0.41458333333333336}


Wikfied Index + Nouns + BM25 + DPH

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/only_nouns.xml",format="trecxml")

def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(wiki_no_difference, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(wiki_no_difference) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()

combined= bm25 + dph
res=combined.transform(topics)



df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]
qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_train.1-8.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10"])

{'map': 0.3033325619464328,
 'ndcg': 0.567624113307805,
 'ndcg_cut_30': 0.407538288088529,
 'P_10': 0.4}

In [ ]:
from pyterrier.measures import *


(Raw Index + QE Approach) + (Wikified Index + PL2 ) + Entities 

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

pl2= pt.BatchRetrieve(wiki_no_difference, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])


qe = pt.BatchRetrieve(raw, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(raw, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(raw, controls={'wmodel': "PL2","c":20})


def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


bm25 = get_retr(raw, 0.75, 1.2) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()
dph = get_dph(raw) >> \
            pt.pipelines.PerQueryMaxMinScoreTransformer()

combined= bm25+dph


gh= qe + pl2 + combined

res=gh.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10","P_5"])

17:09:30.629 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1


{'map': 0.40299498391002503,
 'ndcg': 0.612244241629537,
 'ndcg_cut_30': 0.4405646874031406,
 'P_10': 0.5145833333333335,
 'P_5': 0.5541666666666667}

(Wikified index + QE Approach) + (Raw Index + PL2 ) + Entities 

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

pl2= pt.BatchRetrieve(raw, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])


qe = pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(wiki_no_difference, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(wiki_no_difference, controls={'wmodel': "PL2","c":20})


def get_retr(index, param_b, param_k1):
    retr = pt.BatchRetrieve(index, controls={'wmodel': "BM25", "c": param_b},
        properties={'bm25.k_1': param_k1})
    return retr

def get_dph(index):
    return pt.BatchRetrieve(index, controls={"wmodel": "DPH"})


gh= qe + pl2 

res=gh.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10","P_5"])

17:14:51.592 [main] WARN org.terrier.querying.QueryExpansion - qemodel control not set for QueryExpansion post process. Using default model Bo1


{'map': 0.3990023838148747,
 'ndcg': 0.611758846673892,
 'ndcg_cut_30': 0.45008905770939417,
 'P_10': 0.53125,
 'P_5': 0.5583333333333332}

In [ ]:
topics = pt.io.read_topics("/content/drive/MyDrive/IRDM/ent_test.xml",format="trecxml")

pl2= pt.BatchRetrieve(wiki_no_difference, controls={"wmodel": "PL2","c":20},metadata=["docno",'text'])


qe = pt.BatchRetrieve(raw, controls={'wmodel': "BM25"}) >> \
            pt.rewrite.QueryExpansion(raw, fb_terms=15, fb_docs=6) >> \
            pt.BatchRetrieve(raw, controls={'wmodel': "PL2","c":20})

gh= qe + pl2

res=gh.transform(topics)

df = pd.DataFrame(res, columns=['qid', 'rank', 'docno', 'score'])
df["ep_id"]=df["docno"].apply(lambda x: x.split("_")[1].split("-")[0])
df["episode"]=df["docno"].apply(lambda x: x.split("_")[0])
df["docno"]=df.apply(lambda x: get_ep(x["episode"],x["ep_id"]),axis=1)
df["docno"]=df["docno"]+".0"
rsl=df[["qid","docno","rank","score"]]

qrels_path=("/content/drive/MyDrive/IRDM/podcasts_2020_test.qrels")
qrels = pt.io.read_qrels(qrels_path)
pt.Utils.evaluate(rsl,qrels,metrics=['map',"ndcg","ndcg_cut_30","P_10","P_5"])